****

In [131]:
# Pwsh 7 is not required, I used it to shorten examples
Get-Date | Join-String -f 'Last ran: {0:u}'
$PSVersionTable.PSVersion | Join-String -op 'Pwsh: '

Last ran: 2023-04-16 16:14:21Z
Pwsh: 7.3.0


In [132]:
# generate cached sample data
$LogPath = 'g:\temp\xl\process_list.csv'
# remove-item $LogPath -ea 'ignore' # always recreate?

if(-not (Test-Path $LogPath)) { 
    'Saving Process as csv, this is relatively slow....{0}' -f @( $LogPath )
    Get-Process | Select -first 20 | ConvertTo-Csv | Set-Content -path $LogPath -Encoding utf8
} else {
    $LogPath | Join-String -f 'Loading Cached Version: {0} .'
}

Loading Cached Version: g:\temp\xl\process_list.csv .


In [133]:
function writeColumnSummary {
    # summarize the number of columns, and the names
    param(
        [Parameter(Mandatory)]
        $InputObject
    )

    $Uniques = $InputObject | Sort-Object -unique
    '{0} items => {1}' -f @(
        $Uniques.Count
        $Uniques -join ', ' 
        
    )
}

$contents = Get-Content $Logpath  | ConvertFrom-Csv

# Column names become the PSObject's Property name, so loop on that
$names_all = @($contents)[0].PsObject.Properties.Name

# conditionally select specific columns
$names_64 = $names_all -match '64$' # ends in 64bit

writeColumnSummary $names_all
writeColumnSummary $names_64



69 items => __NounName, BasePriority, CommandLine, Company, Container, CPU, Description, EnableRaisingEvents, ExitCode, ExitTime, FileVersion, Handle, HandleCount, Handles, HasExited, Id, MachineName, MainModule, MainWindowHandle, MainWindowTitle, MaxWorkingSet, MinWorkingSet, Modules, Name, NonpagedSystemMemorySize, NonpagedSystemMemorySize64, NPM, PagedMemorySize, PagedMemorySize64, PagedSystemMemorySize, PagedSystemMemorySize64, Parent, Path, PeakPagedMemorySize, PeakPagedMemorySize64, PeakVirtualMemorySize, PeakVirtualMemorySize64, PeakWorkingSet, PeakWorkingSet64, PM, PriorityBoostEnabled, PriorityClass, PrivateMemorySize, PrivateMemorySize64, PrivilegedProcessorTime, ProcessName, ProcessorAffinity, Product, ProductVersion, Responding, SafeHandle, SessionId, SI, Site, StandardError, StandardInput, StandardOutput, StartInfo, StartTime, SynchronizingObject, Threads, TotalProcessorTime, UserProcessorTime, VirtualMemorySize, VirtualMemorySize64, VM, WorkingSet, WorkingSet64, WS
9 item

In [134]:
# ## maybe final
# $col = ''
# foreach ($column in $cols_edispensary) { 
#     if ($column.Substring(0, $column.Length - 2) -ne 'Blank') {
#         $col = $col + ', ' + $column.ToString()
#     }
# } # drop all column names that begin with 'Blank'

# $col = $col.Substring(2, $col.Length - 2) # Take away the initial comma and space at the start.

$column_schema = @{
    Text   = 'FirstName'
    Number = 'PhoneNumber'
    # '...'
}
$path_csv = (Join-Path '.' 'Data/edispensary.csv')

$wanted_columns = @($cols_edispensary)[0].PsObject.Properties.Name
| ?{ $_.Name -notmatch '^blank' } # ignore anything that is prefixed with 'blank'

$importDbaCsvSplat = @{
    Path        = Get-Item -ea 'stop' $path_csv
    Delimiter   = ','
    SqlInstance = $SQLserver
    Database    = $SQLdatabase
    Schema      = $SQLSchema
    Table       = 'edispensary'
    # Column      = $col
    ColumnMap = $column_schema
    Truncate    = $true
}

$PathCsv = Get-Item -ea 'stop' $path_csv
$records = Get-Content -path $PathCsv | ConvertFrom-Csv 

# Import-DbaCsv @importDbaCsvSplat


# $importDbaCsvSplat = @{
#     Path        = 'c:/temp/supersmall.csv'
#     Database    = 'tempdb'
#     SqlInstance = 'sql2016'
#     ColumnMap   = $columns
# }

# Import-DbaCsv @importDbaCsvSplat

Get-Item: 
Line |
  22 |      Path        = Get-Item -ea 'stop' $path_csv
     |                    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Cannot find path 'H:\data\2023\pwsh\notebooks\sketches\ask_queue\Data\edispensary.csv' because it does not exist.


## Original code from a Discord thread

- [discord](https://discord.com/channels/180528040881815552/800214699463409704/1096803145096630345)

```ps1
$col = ''
foreach ($column in $cols_edispensary) { 
    if ($column.Substring(0, $column.Length - 2) -ne 'Blank') {
        $col = $col + ', ' + $column.ToString()
    }
} # drop all column names that begin with 'Blank'

$col = $col.Substring(2, $col.Length - 2) # Take away the initial comma and space at the start.
$importDbaCsvSplat = @{
    Path        = Join-Path $folder_cleaned 'Data/edispensary.csv'
    Delimiter   = ','
    SqlInstance = $SQLserver
    Database    = $SQLdatabase
    Schema      = $SQLSchema
    Table       = 'edispensary'
    Column      = $col
    Truncate    = $true
}

Import-DbaCsv @importDbaCsvSplat

$columns = @{
    Text   = 'FirstName'
    Number = 'PhoneNumber'
}
$importDbaCsvSplat = @{
    Path        = 'c:/temp/supersmall.csv'
    SqlInstance = 'sql2016'
    Database    = 'tempdb'
    ColumnMap   = $columns
}

Import-DbaCsv @importDbaCsvSplat
```

# About: EditorServicesCommandSuite

To enable the amazing hotkey `ConvertTo-SplatExpression` and other hotkeys, you can view this thread. It's the sticky in the `#vscode` channel. <https://discord.com/channels/180528040881815552/446531919644065804/1005277211659604059>


## Step1: Install the module:

```ps1
Install-Module -Scope CurrentUser -AllowPrerelease EditorServicesCommandSuite
```

## Step2: Update your profile

In your profile, enable automatic loading using this command. To mport only in VsCode, save it in your profile at the filepath `$PROFILE.AllUsersCurrentHost` . 
If you want your profile to always load -- inside and outside of vs code, then edit the filepath `$PROFILE.CurrentUserAllHosts`

```ps1
Import-CommandSuite
```

## Step3: New Keybindings

```jsonc
    {
        "key": "ctrl+.",
        "command": "PowerShell.InvokeRegisteredEditorCommand",
        "args": { "commandName": "Invoke-DocumentRefactor" },
        "when": "editorLangId == 'powershell'"
    },
    {
        "key": "ctrl+shift+s",
        "command": "PowerShell.InvokeRegisteredEditorCommand",
        "args": { "commandName": "ConvertTo-SplatExpression" },
        "when": "editorLangId == 'powershell'"
    },
```